# 🛍️ Retail Sales ETL & Analysis Project

This notebook demonstrates a complete ETL (Extract, Transform, Load) process on retail sales data for January and February. It includes SQL-style queries and visualizations for insight generation.

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")


## 📥 Step 1: Extract Data

In [ ]:
sales_jan = pd.read_csv('sales_jan.csv')
sales_feb = pd.read_csv('sales_feb.csv')
products = pd.read_csv('products.csv')

# Combine sales data
sales = pd.concat([sales_jan, sales_feb])
sales.head()

## 🔄 Step 2: Transform Data

In [ ]:
# Calculate total amount
sales['TotalAmount'] = sales['Quantity'] * sales['Price']

# Merge with product info
sales = sales.merge(products, on='ProductID')

# Convert date column
sales['Date'] = pd.to_datetime(sales['Date'])
sales.head()

## 💾 Step 3: Load into SQLite Database

In [ ]:
conn = sqlite3.connect('retail.db')
sales.to_sql('sales_data', conn, if_exists='replace', index=False)


## 📊 Step 4: SQL Queries for Insights

In [ ]:
query = '''
SELECT strftime('%Y-%m', Date) AS Month, SUM(TotalAmount) AS Revenue
FROM sales_data
GROUP BY Month
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT ProductName, SUM(TotalAmount) AS TotalRevenue
FROM sales_data
GROUP BY ProductName
ORDER BY TotalRevenue DESC
LIMIT 5
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT Category, SUM(Quantity) AS TotalSold
FROM sales_data
GROUP BY Category
ORDER BY TotalSold DESC
'''
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT AVG(TotalAmount) AS AvgOrderValue
FROM sales_data
'''
pd.read_sql(query, conn)

## 📈 Step 5: Visualizations

In [ ]:
# Monthly revenue bar chart
monthly_sales = sales.groupby(sales['Date'].dt.to_period('M'))['TotalAmount'].sum()
monthly_sales.plot(kind='bar', title='Monthly Sales Revenue', ylabel='Revenue', xlabel='Month')
plt.show()


In [ ]:
# Category-wise sales pie chart
category_sales = sales.groupby('Category')['TotalAmount'].sum()
category_sales.plot(kind='pie', autopct='%1.1f%%', title='Sales by Category')
plt.ylabel('')
plt.show()


## 🧠 Step 6: Business Insights & Recommendations

- **Insight 1:** Revenue increased in February, mainly driven by smartphone sales.
- **Insight 2:** Electronics dominate the revenue share.
- **Recommendation:** Focus marketing efforts on top-performing products and consider bundling accessories like chargers with smartphones.